In [24]:
import sys
import json

params = {'general':{'periods':['test']}}
default = {'scenario':'orchestrator','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'scenario': 'orchestrator', 'training_folder': '../..', 'params': {'general': {'periods': ['test']}}}


In [25]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [26]:
periods = argv['params']['general']['periods']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'
if on_lambda:
    bucket_name = 'quetzal-immense' #os.environ.get('BUCKET_NAME')
    output_folder = training_folder + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = 's3://' + bucket_name + '/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')

else:
    output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = training_folder + '/scenarios/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')
    
print('input folder: ', input_folders)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folders)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ['../../inputs/test/']
output folder:  ../../scenarios/orchestrator/outputs/
scen folder :  ['../../scenarios/test/outputs/']


In [27]:
#var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# modal shift

In [28]:
def get_pt_part(pt_los, car_los, volumes, method='kirchhoff', power=-1.42, part_max=1):
    pt_time = pt_los.set_index(['origin','destination'])['time'].to_dict()
    car_time = car_los.set_index(['origin','destination'])['time'].to_dict()
    volumes['pt_time'] = volumes.set_index(['origin','destination']).index.map(pt_time.get)
    volumes['car_time'] = volumes.set_index(['origin','destination']).index.map(car_time.get)
    volumes['ratio_temps'] = volumes['pt_time'] / volumes['car_time']
    if method == 'kirchhoff':
        volumes['part_pt'] = volumes['ratio_temps'].apply(lambda x: min(x**(power), part_max))
    else:
        print('method not recognized')
    return volumes


In [29]:
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(ref_folder)
    ref_pt_skim = pd.read_csv(ref_folder+'skims/pt_skim.csv')
    ref_car_skim = pd.read_csv(ref_folder+'skims/car_skim.csv')
    ref_volumes = pd.read_csv(ref_folder+'volumes/volumes.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    car_skim = pd.read_csv(scen_folder+'car_skim.csv')
    
    ref_pt_part = get_pt_part(ref_pt_skim, ref_car_skim, ref_volumes.copy())
    pt_part = get_pt_part(pt_skim, car_skim, ref_volumes.copy())
    
    ref_pt_part = ref_pt_part.set_index(['origin','destination'])['part_pt'].to_dict()
    pt_part['ref_part_pt'] = pt_part.set_index(['origin','destination']).index.map(ref_pt_part.get)
    pt_part['probabilite transfert'] = pt_part['part_pt'] - pt_part['ref_part_pt']
    transfert = pt_part['pt'] * pt_part['probabilite transfert']
    pt_part['pt'] = pt_part['pt'] + transfert
    pt_part['car'] = pt_part['car'] - transfert
    
    pt_part[ref_volumes.columns].to_csv(output_folder+'volume_'+period+'.csv')

../../inputs/test/
